In [1]:
import numpy as np
import cv2
import json
import os
import pickle

In [2]:
f = open('labels/Labels_JSON.json')
data = json.load(f)
images = data["images"]

In [3]:
f = open('labels/Labels_JSON.json')
data = json.load(f)
images = data["images"]
labels = []
image_names = []

for image in images:
    image_name = int(image['image_name'][:-4])
    image_names.append(image_name)
    label = image['labels']
    labels.append(label)

labels[12]

def rle_decode(mask_rle, shape):
    """
    mask_rle: run-length as string formatted (start length)
    shape: (width, height) of array to return
    Returns numpy array, 1 - mask, 0 - background
    """
    s = mask_rle
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    shape = shape[1], shape[0]
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 255
    return img.reshape(shape)

In [4]:
def get_filepaths(directory,low,high,exten='.jpg'):
    
    filepaths = []
    for filename in os.listdir(directory):
        basename, ext = os.path.splitext(filename)
        if ext != exten:
            continue
        try:
            number = int(basename)
        except ValueError:
            continue  # not numeric
        if low <= number <= high:
            # process file
            filename = os.path.join(directory, filename)
            filepaths.append(basename)
            
    return filepaths

In [5]:
label_image_pair = {}
for image in images:
    image_name = int(image['image_name'][:-4])
    label = image['labels']
    for l in label: 
        if l['class_name'] in label_image_pair.keys():
            label_image_pair[l['class_name']].append(image_name)
        else:
            label_image_pair[l['class_name']] = [image_name]

In [6]:
image_label_pair = {}
for image in images:
    image_name = int(image['image_name'][:-4])
    label = image['labels']
    for l in label: 
        if image_name in image_label_pair.keys():
            image_label_pair[image_name].append(l['class_name'])
        else:
            image_label_pair[image_name] = [l['class_name']]

In [7]:
with open(f'l_to_i.pyb', "wb") as fp:   #Pickling
    pickle.dump(label_image_pair, fp)
with open(f'i_l.pyb', "wb") as fp:   #Pickling
    pickle.dump(image_label_pair, fp)

In [13]:
shape = (1500, 3000, 3)
mask = np.zeros(shape, dtype = np.uint8)
i = 0
masks = []
color13 = np.array([255, 255, 255])
#color13 = np.array([222, 222, 55])
numbers = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
#colors = [(50, 69, 57), (170, 98, 195), (222, 222, 55), (112, 168, 172), (112, 215, 214)] # For reducing number of classes
colors = [(50, 69, 57), (170, 98, 195), (222, 222, 55), (112, 168, 172), (112, 215, 214), (181, 72, 24), (22, 132, 221), (167, 228, 28), (124, 33, 204), (16, 0, 241), (72, 117, 136), (70, 113, 190), (149, 89, 6), (255, 255, 255)] # For all classes
#colors = [(0, 0, 0), (50, 69, 57), (50, 69, 57), (50, 69, 57), (50, 69, 57), (50, 69, 57), (22, 132, 221), (167, 228, 28), (124, 33, 204), (16, 0, 241), (72, 117, 136), (70, 113, 190), (149, 89, 6), (255, 255, 255)] # All Crease as same class
#colors = [(0, 0, 0), (50, 69, 57), (50, 69, 57), (50, 69, 57), (50, 69, 57), (50, 69, 57), (170, 98, 195), (170, 98, 195), (170, 98, 195), (222, 222, 55), (222, 222, 55), (222, 222, 55), (222, 222, 55), (222, 222, 55)]

ll = 0 
for label in labels:
    
    if image_names[ll] in np.arange(0,600):
#         print(image_names[ll])
        ll += 1
        continue
    
    visited_classes = []
    
    for value in label:
        
        mask = np.zeros(shape, dtype = np.uint8)
        class_name = int(value['class_name'])
        
        if class_name == 3:
            continue
        
        visited_classes.append(class_name)
            
#         print('here')
        
        if(value['polygon'] != None):

            points = np.array([value['polygon']], dtype = np.int32)
            cv2.fillPoly(mask, points, color = colors[class_name])
        else:
            
            brush = np.zeros(shape, dtype = np.uint8)
            bbox = value['bbox']
            xyz = rle_decode(value['mask'], (bbox[2]-bbox[0], bbox[3]-bbox[1]))
            xyz = cv2.merge([xyz, xyz, xyz])
            brush[bbox[1]:bbox[3], bbox[0]:bbox[2]] = xyz
            
            for i in range(brush.shape[0]):
                for j in range(brush.shape[1]):
                    if((brush[i, j] == color13).all()):
                        
                        mask[i, j] = colors[class_name]
            
    
#         x = cv2.resize(mask, dsize=(3000, 1500), interpolation = cv2.INTER_NEAREST)
        cv2.imwrite(f'../dataset/mask_1500x3000/{image_names[ll]}_{class_name}.jpg', mask)
    
#     for neg_class in set(np.arange(1,14)) - set([3]) - set(visited_classes):
#         mask = np.zeros((256, 512,3), dtype = np.uint8)
#         cv2.imwrite(f'../labels/masks_c/{image_names[ll]}_{neg_class}.jpg', mask)

    ll += 1

In [30]:
set(np.arange(1,14)) - set([3]) - set(visited_classes)

{1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13}

In [21]:
for i in set(np.arange(1,14)) - set([3]) - set(visited)):
    print(i)

1
2
3
4
5
9
10
11
12
13


In [41]:
np.where([0,1,1,0,0,1,0] == 1)[0]

array([], dtype=int64)

In [42]:
[0,1,1,0,0,1,0] == 1

False

In [6]:
len(colors)

14

In [7]:
for i in range(len(masks)):
    
#     x = cv2.resize(masks[i], dsize=(512, 256), interpolation = cv2.INTER_NEAREST)
    #print(image_names[i])
#     cv2.imshow('mask', x)
#     cv2.waitKey(10)
# cv2.destroyAllWindows()